In [17]:
import pandas as pd
import os
import FinanceDataReader as fdr

# CSV 파일 경로 설정
file_path = "./Data/kopsi_stocks.csv"

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 'Code'와 'Company' 열에서 중복 제거
unique_codes = df[['Code', 'Company']].drop_duplicates()

# "Data/Stock" 폴더 생성
folder_path = "Data/Stock"
os.makedirs(folder_path, exist_ok=True)

# 각 종목 코드에 대해 주가 데이터 수집 및 업데이트
for _, row in unique_codes.iterrows():
    code = row['Code'].replace("'", "")  # 종목 코드에서 작은따옴표 제거
    company = row['Company']             # 회사명 가져오기
    
    file_name = f"{company}_Stock.csv"
    file_path = os.path.join(folder_path, file_name)
    
    try:
        # 새 데이터를 수집
        new_data = fdr.DataReader(code, start='2021-01-01', end='2023-12-31')
        
        # Change 열 계산 (Open 대비 Close 변화율)
        new_data['Change'] = ((new_data['Close'] - new_data['Open']) / new_data['Open']) * 100
        
        # 모든 날짜 포함 (일요일 포함)
        all_dates = pd.date_range(start='2021-01-01', end='2023-12-31', freq='D')
        new_data = new_data.reindex(all_dates)
        
        # 인덱스를 Date 열로 변환
        new_data.index.name = 'Date'  # 인덱스 이름을 Date로 지정
        new_data.reset_index(inplace=True)  # 인덱스를 열로 변환
        
        # 'Next_Day_Change' 채우기
        new_data['Next_Day_Change'] = None
        new_data['Change'] = new_data['Change'].fillna(0)  # 영업일이 아닌 날은 Change=0으로 처리
        
        # 영업일 기준으로 'Next_Day_Change' 채우기
        for idx in range(len(new_data) - 1):
            if pd.notna(new_data.iloc[idx]['Change']):
                for j in range(idx + 1, len(new_data)):
                    if pd.notna(new_data.iloc[j]['Change']):
                        new_data.iloc[idx, new_data.columns.get_loc('Next_Day_Change')] = new_data.iloc[j]['Change']
                        break
        
        # 파일이 이미 존재하는 경우, 기존 데이터와 병합
        if os.path.exists(file_path):
            existing_data = pd.read_csv(file_path, parse_dates=['Date'])
            
            # 기존 데이터의 Change 및 Next_Day_Change 업데이트
            existing_data['Change'] = ((existing_data['Close'] - existing_data['Open']) / existing_data['Open']) * 100
            existing_data = existing_data.set_index('Date').reindex(all_dates).reset_index()
            existing_data['Next_Day_Change'] = None
            existing_data['Change'] = existing_data['Change'].fillna(0)
            
            # 영업일 기준으로 'Next_Day_Change' 채우기
            for idx in range(len(existing_data) - 1):
                if pd.notna(existing_data.iloc[idx]['Change']):
                    for j in range(idx + 1, len(existing_data)):
                        if pd.notna(existing_data.iloc[j]['Change']):
                            existing_data.iloc[idx, existing_data.columns.get_loc('Next_Day_Change')] = existing_data.iloc[j]['Change']
                            break
            
            updated_data = pd.concat([existing_data, new_data]).drop_duplicates(subset=['Date']).sort_values(by='Date')
        else:
            updated_data = new_data
        
        # 병합된 데이터를 다시 CSV로 저장
        updated_data.to_csv(file_path, index=False, encoding='utf-8-sig')
        print(f"{company} ({code}) stock data with updated Change and Next_Day_Change saved to {file_path}")
    except Exception as e:
        print(f"Failed to fetch stock data for {company} ({code}): {str(e)}")

print("Stock data collection with Change and Next_Day_Change update completed.")


삼성전자 (005930) stock data with updated Change and Next_Day_Change saved to Data/Stock\삼성전자_Stock.csv
SK하이닉스 (000660) stock data with updated Change and Next_Day_Change saved to Data/Stock\SK하이닉스_Stock.csv
LG에너지솔루션 (373220) stock data with updated Change and Next_Day_Change saved to Data/Stock\LG에너지솔루션_Stock.csv
삼성바이오로직스 (207940) stock data with updated Change and Next_Day_Change saved to Data/Stock\삼성바이오로직스_Stock.csv
현대차 (005380) stock data with updated Change and Next_Day_Change saved to Data/Stock\현대차_Stock.csv
기아 (000270) stock data with updated Change and Next_Day_Change saved to Data/Stock\기아_Stock.csv
셀트리온 (068270) stock data with updated Change and Next_Day_Change saved to Data/Stock\셀트리온_Stock.csv
KB금융 (105560) stock data with updated Change and Next_Day_Change saved to Data/Stock\KB금융_Stock.csv
NAVER (035420) stock data with updated Change and Next_Day_Change saved to Data/Stock\NAVER_Stock.csv
신한지주 (055550) stock data with updated Change and Next_Day_Change saved to Data/Stock

In [21]:
import pandas as pd
import os

# Data/Stock 폴더 경로 설정
folder_path = "Data/Stock"
output_path = os.path.join(folder_path, "Kospi.csv")

# Data/Stock 폴더에 있는 모든 CSV 파일 목록 가져오기 (Kospi.csv 제외)
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f != "Kospi.csv"]

# 모든 CSV 파일을 읽어 Date별 'Change' 및 'Next_Day_Change' 평균 계산
all_data = pd.DataFrame()

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(file_path)
        
        # 필수 열이 있는지 확인
        if 'Date' in df.columns and 'Change' in df.columns and 'Next_Day_Change' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # Date 열을 datetime 형식으로 변환
            df = df.dropna(subset=['Date', 'Change', 'Next_Day_Change'])  # 유효하지 않은 값 제거
                      
            # 필요한 열만 추출
            df = df[['Date', 'Change', 'Next_Day_Change']]
            all_data = pd.concat([all_data, df], ignore_index=True)
    except Exception as e:
        print(f"Failed to process file {file}: {str(e)}")

# 데이터가 비어 있지 않은지 확인
if not all_data.empty:
    # Date별로 Change와 Next_Day_Change의 평균 계산
    average_change = all_data.groupby('Date').agg({
        'Change': 'mean',
        'Next_Day_Change': 'mean'
    }).reset_index()
    average_change.rename(columns={
        'Change': 'Change',
        'Next_Day_Change': 'Next_Day_Change'
    }, inplace=True)

    # 결과를 CSV 파일로 저장
    average_change.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Date별 평균 Change 및 Next_Day_Average_Change가 {output_path}에 저장되었습니다.")
else:
    print("No valid data to process. Check the input files.")


Date별 평균 Change 및 Next_Day_Average_Change가 Data/Stock\Kospi.csv에 저장되었습니다.
